In [1]:
import statsmodels.api as sm
from shapely.geometry import Point, Polygon
import geopandas as gpd
import pandas as pd
import numpy as np
import os
import geopandas as gpd

# import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', 500)
pd.options.display.max_rows = 1000

plt.style.use('seaborn-darkgrid')


from statsmodels.tsa.api import ARDL

In [2]:
input_dir = '../../data/final'

df_final = pd.read_csv(os.path.join(input_dir, 'df_merged_inner_518.csv'))

/var/folders/38/ttqg2y215g16g2ng7jd502_c0000gn/T/ipykernel_11872/1605361495.py:3: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_final = pd.read_csv(os.path.join(input_dir, 'df_merged_inner_518.csv'))


In [3]:
df_final['wind_speed'] = np.sqrt(df_final['u10']**2 + df_final['v10']**2)
df_final[['wind_speed', 't2m', 'tp']].isnull().sum()

# df_final.to_csv(os.path.join(input_dir, 'df_final.csv'), index=False)

wind_speed    0
t2m           0
tp            0
dtype: int64

In [4]:
df_final.head()

,Unnamed: 0,index_x,datetime,Location name,ESMI_ID,District,State,Category,Connection Type,Lat,Lon,t2m,u10,v10,tp,date,time,station_id,index_y,pct_blackout,wind_speed
0,0,2569489,2014-11-17 17:00:00,Vijay Nagar,NaN,Hyderabad,Telangana,State Capital,Domestic,17.394295,78.451016,295.09723,-1.271197,-0.320305,5.645638e-04,20141117,1700,495,7239294,0.0,1.310930
1,1,2569493,2014-11-29 17:00:00,Vijay Nagar,NaN,Hyderabad,Telangana,State Capital,Domestic,17.394295,78.451016,293.04870,-1.548639,-1.584031,0.000000e+00,20141129,1700,495,7239582,0.0,2.215274
2,2,2569500,2014-11-30 22:00:00,Vijay Nagar,NaN,Hyderabad,Telangana,State Capital,Domestic,17.394295,78.451016,289.83447,-1.289795,-1.528702,4.351137e-07,20141130,2200,495,7239611,0.0,2.000125
3,3,2569503,2014-11-13 07:00:00,Vijay Nagar,NaN,Hyderabad,Telangana,State Capital,Domestic,17.394295,78.451016,297.14110,-3.871719,0.472760,6.666072e-04,20141113,700,495,7239188,0.0,3.900476
4,4,2569506,2014-12-31 15:00:00,Vijay Nagar,NaN,Hyderabad,Telangana,State Capital,Domestic,17.394295,78.451016,296.46838,-2.901352,0.717420,2.732359e-06,20141231,1500,495,7240348,0.0,2.988735


In [8]:

geo_df = gpd.read_file('../../data/helper/station_forest_cover.geojson')

# check climate_zone_code values
geo_df['climate_zone_code'].unique()

# drop rows with missing climate_zone_code
geo_df = geo_df[geo_df['climate_zone_code'].notnull()]

# add a column for the second level of climate zone
# reclassify the climate_zone_code for the second level
# 'Cwb', 'Cwa' -> 'Cw'
# 'Bsh' -> 'BS'
# 'Aw' -> 'Aw'
# 'Am' -> 'Am'
# 'Dwb' -> 'Dw'

geo_df['climate_zone_code_2'] = geo_df['climate_zone_code'].replace({'Cwb': 'Cw', 'Cwa': 'Cw', 'BSh': 'BSh', 'Aw': 'Aw', 'Am': 'Am', 'Dwb': 'Dw'})
# print the unique values of the new column
geo_df['climate_zone_code_2'].unique()

# print the count of each unique value
geo_df['climate_zone_code_2'].value_counts()

Cw     250
Aw     141
BSh    115
Am      28
Dw       1
Name: climate_zone_code_2, dtype: int64

In [9]:

df_final_geo = pd.merge(df_final, geo_df, on='station_id', how='left')

df_final_geo.head()



,Unnamed: 0,index_x,datetime,Location name_x,ESMI_ID_x,District,State,Category_x,Connection Type_x,Lat_x,Lon_x,t2m,u10,v10,tp,date,time,station_id,index_y,pct_blackout,wind_speed,Location name_y,ESMI_ID_y,Category_y,Connection Type_y,Lat_y,Lon_y,climate_zone_DN,climate_zone_name,climate_zone_code,tree2000,gain,lossyear,geometry,climate_zone_code_2
0,0,2569489,2014-11-17 17:00:00,Vijay Nagar,NaN,Hyderabad,Telangana,State Capital,Domestic,17.394295,78.451016,295.09723,-1.271197,-0.320305,5.645638e-04,20141117,1700,495,7239294,0.0,1.310930,Vijay Nagar,NaN,State Capital,Domestic,17.394295,78.451016,3.0,"Tropical, savannah",Aw,0.0,0.0,2000.0,POINT (78.45102 17.39429),Aw
1,1,2569493,2014-11-29 17:00:00,Vijay Nagar,NaN,Hyderabad,Telangana,State Capital,Domestic,17.394295,78.451016,293.04870,-1.548639,-1.584031,0.000000e+00,20141129,1700,495,7239582,0.0,2.215274,Vijay Nagar,NaN,State Capital,Domestic,17.394295,78.451016,3.0,"Tropical, savannah",Aw,0.0,0.0,2000.0,POINT (78.45102 17.39429),Aw
2,2,2569500,2014-11-30 22:00:00,Vijay Nagar,NaN,Hyderabad,Telangana,State Capital,Domestic,17.394295,78.451016,289.83447,-1.289795,-1.528702,4.351137e-07,20141130,2200,495,7239611,0.0,2.000125,Vijay Nagar,NaN,State Capital,Domestic,17.394295,78.451016,3.0,"Tropical, savannah",Aw,0.0,0.0,2000.0,POINT (78.45102 17.39429),Aw
3,3,2569503,2014-11-13 07:00:00,Vijay Nagar,NaN,Hyderabad,Telangana,State Capital,Domestic,17.394295,78.451016,297.14110,-3.871719,0.472760,6.666072e-04,20141113,700,495,7239188,0.0,3.900476,Vijay Nagar,NaN,State Capital,Domestic,17.394295,78.451016,3.0,"Tropical, savannah",Aw,0.0,0.0,2000.0,POINT (78.45102 17.39429),Aw
4,4,2569506,2014-12-31 15:00:00,Vijay Nagar,NaN,Hyderabad,Telangana,State Capital,Domestic,17.394295,78.451016,296.46838,-2.901352,0.717420,2.732359e-06,20141231,1500,495,7240348,0.0,2.988735,Vijay Nagar,NaN,State Capital,Domestic,17.394295,78.451016,3.0,"Tropical, savannah",Aw,0.0,0.0,2000.0,POINT (78.45102 17.39429),Aw


In [10]:
# drop columns: Loctionname_y, ESMI_ID_y, Category_y, Lat_y, Lon_y, geometry_y
df_final_geo.drop(columns=['Location name_y', 'ESMI_ID_y',
                  'Category_y', 'Lat_y', 'Lon_y'], inplace=True)
# rename: Loctionname_x -> Loctionname, ESMI_ID_x -> ESMI_ID, Category_x -> Category, Lat_x -> Lat, Lon_x -> Lon
df_final_geo.rename(columns={'Loction name_x': 'Loctionname', 'ESMI_ID_x': 'ESMI_ID', 'Category_x': 'Category', 'Lat_x': 'Lat', 'Lon_x': 'Lon'}, inplace=True)

df_final_geo.to_csv(os.path.join(input_dir, 'df_final_geo_518.csv'), index=False)

In [ ]:

# drop the rows with Dw in climate_zone_code_2
df_final_geo = df_final_geo[df_final_geo['climate_zone_code_2'] != 'Dw']

df_final_geo['climate_zone_code_2'].value_counts()


Aw     2440752
Cw     2353832
BSh    1770277
Am      479518
Name: climate_zone_code_2, dtype: int64

In [12]:

df_final_geo.to_csv(os.path.join(
    input_dir, 'df_final_geo_518.csv'), index=False)


In [14]:
# add a column for wind*forest_cover

df_final_geo['wind_forest_cover'] = df_final_geo['wind_speed'] * \
    df_final_geo['tree2000']
    
df_final_geo.to_csv(os.path.join(
    input_dir, 'df_final_geo_518.csv'), index=False)

df_final_geo.head()

,Unnamed: 0,index_x,datetime,Location name_x,ESMI_ID,District,State,Category,Connection Type_x,Lat,Lon,t2m,u10,v10,tp,date,time,station_id,index_y,pct_blackout,wind_speed,Connection Type_y,climate_zone_DN,climate_zone_name,climate_zone_code,tree2000,gain,lossyear,geometry,climate_zone_code_2,wind_forest_cover
0,0,2569489,2014-11-17 17:00:00,Vijay Nagar,NaN,Hyderabad,Telangana,State Capital,Domestic,17.394295,78.451016,295.09723,-1.271197,-0.320305,5.645638e-04,20141117,1700,495,7239294,0.0,1.310930,Domestic,3.0,"Tropical, savannah",Aw,0.0,0.0,2000.0,POINT (78.45102 17.39429),Aw,0.0
1,1,2569493,2014-11-29 17:00:00,Vijay Nagar,NaN,Hyderabad,Telangana,State Capital,Domestic,17.394295,78.451016,293.04870,-1.548639,-1.584031,0.000000e+00,20141129,1700,495,7239582,0.0,2.215274,Domestic,3.0,"Tropical, savannah",Aw,0.0,0.0,2000.0,POINT (78.45102 17.39429),Aw,0.0
2,2,2569500,2014-11-30 22:00:00,Vijay Nagar,NaN,Hyderabad,Telangana,State Capital,Domestic,17.394295,78.451016,289.83447,-1.289795,-1.528702,4.351137e-07,20141130,2200,495,7239611,0.0,2.000125,Domestic,3.0,"Tropical, savannah",Aw,0.0,0.0,2000.0,POINT (78.45102 17.39429),Aw,0.0
3,3,2569503,2014-11-13 07:00:00,Vijay Nagar,NaN,Hyderabad,Telangana,State Capital,Domestic,17.394295,78.451016,297.14110,-3.871719,0.472760,6.666072e-04,20141113,700,495,7239188,0.0,3.900476,Domestic,3.0,"Tropical, savannah",Aw,0.0,0.0,2000.0,POINT (78.45102 17.39429),Aw,0.0
4,4,2569506,2014-12-31 15:00:00,Vijay Nagar,NaN,Hyderabad,Telangana,State Capital,Domestic,17.394295,78.451016,296.46838,-2.901352,0.717420,2.732359e-06,20141231,1500,495,7240348,0.0,2.988735,Domestic,3.0,"Tropical, savannah",Aw,0.0,0.0,2000.0,POINT (78.45102 17.39429),Aw,0.0


In [15]:
df_final_geo.columns


Index(['Unnamed: 0', 'index_x', 'datetime', 'Location name_x', 'ESMI_ID',
       'District', 'State', 'Category', 'Connection Type_x', 'Lat', 'Lon',
       't2m', 'u10', 'v10', 'tp', 'date', 'time', 'station_id', 'index_y',
       'pct_blackout', 'wind_speed', 'Connection Type_y', 'climate_zone_DN',
       'climate_zone_name', 'climate_zone_code', 'tree2000', 'gain',
       'lossyear', 'geometry', 'climate_zone_code_2', 'wind_forest_cover'],
      dtype='object')

In [17]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm


In [18]:


from statsmodels.tsa.ardl import ARDL
df_final_geo['climate_zone_code_2_cat'] = df_final_geo['climate_zone_code_2'].astype('category')
df_final_geo['climate_zone_code_2_cat'] = df_final_geo['climate_zone_code_2_cat'].cat.codes

df_final_geo['climate_zone_code_2_cat'].value_counts()

df_final_geo['climate_zone_code_2_cat'].unique()

array([ 1,  2,  3,  0, -1], dtype=int8)

In [ ]:
df_final_geo.plot(figsize=(12, 8))

<AxesSubplot:>

In [ ]:
from statsmodels.tsa.stattools import adfuller


def adf_test(timeseries):
    print("Results of Dickey-Fuller Test:")
    dftest = adfuller(timeseries, autolag="AIC")
    dfoutput = pd.Series(
        dftest[0:4],
        index=[
            "Test Statistic",
            "p-value",
            "#Lags Used",
            "Number of Observations Used",
        ],
    )
    for key, value in dftest[4].items():
        dfoutput["Critical Value (%s)" % key] = value
    print(dfoutput)

In [43]:
from statsmodels.tsa.ardl import ardl_select_order

test_model_all = ardl_select_order(df_final_geo['pct_blackout'],
                                       exog=df_final_geo[[
                                           't2m', 'wind_speed', 'tp']],
                                       maxlag=4, ic='aic',maxorder=4)

print(test_model_all.summary())




/Users/rainylty/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/rainylty/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


AttributeError: 'ARDLOrderSelectionResults' object has no attribute 'summary'

In [41]:
import matplotlib.pyplot as plt

# Calculate Impulse Responses
irf = test_model_all.irf(10)
irf.plot(orth=False, impulse='t2m')
plt.show()



AttributeError: 'ARDLResults' object has no attribute 'irf'

In [20]:
from statsmodels.tsa.ardl import ardl_select_order
import matplotlib.pyplot as plt

# Get unique climate zones
climate_zones = df_final_geo['climate_zone_code_2'].unique()

# Dictionary to store ARDL models and summaries for each climate zone
climate_zone_models = {}

# Loop through each climate zone
for zone in climate_zones:
    # Filter the data for the current climate zone
    climate_zone_data = df_final_geo[df_final_geo['climate_zone_code_2'] == zone]

    # Select the optimal ARDL model based on AIC for this climate zone
    selected_model = ardl_select_order(climate_zone_data['pct_blackout'],
                                       exog=climate_zone_data[[
                                           't2m', 'wind_speed', 'tp']],
                                       maxlag=4, ic='aic',maxorder=4)

    # Fit the ARDL model with the selected lags
    model = selected_model.model.fit()

    # Store the model summary for later use
    climate_zone_models[zone] = model

    # Print out or log the summary for this climate zone
    print(f"\nClimate Zone: {zone}")
    print(model.summary())

/Users/rainylty/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/rainylty/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)



Climate Zone: Aw
                              ARDL Model Results                              
Dep. Variable:           pct_blackout   No. Observations:              2440752
Model:               ARDL(4, 4, 4, 4)   Log Likelihood             1219509.056
Method:               Conditional MLE   S.D. of innovations              0.147
Date:                Thu, 24 Oct 2024   AIC                       -2438976.112
Time:                        13:03:31   BIC                       -2438709.248
Sample:                             4   HQIC                      -2438905.200
                              2440752                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.1559      0.011     13.632      0.000       0.133       0.178
pct_blackout.L1     0.0453      0.001     70.808      0.000       0.044       0.047
pct_blackout.L

/Users/rainylty/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/rainylty/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)



Climate Zone: BSh
                              ARDL Model Results                              
Dep. Variable:           pct_blackout   No. Observations:              1770277
Model:               ARDL(4, 4, 4, 4)   Log Likelihood               71919.125
Method:               Conditional MLE   S.D. of innovations              0.232
Date:                Thu, 24 Oct 2024   AIC                        -143796.250
Time:                        13:17:13   BIC                        -143536.130
Sample:                             4   HQIC                       -143726.265
                              1770277                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.4358      0.018    -24.027      0.000      -0.471      -0.400
pct_blackout.L1     0.1483      0.001    199.798      0.000       0.147       0.150
pct_blackout.

/Users/rainylty/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/rainylty/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)



Climate Zone: Cw
                              ARDL Model Results                              
Dep. Variable:           pct_blackout   No. Observations:              2353832
Model:               ARDL(4, 4, 4, 4)   Log Likelihood             -757505.175
Method:               Conditional MLE   S.D. of innovations              0.334
Date:                Thu, 24 Oct 2024   AIC                        1515052.350
Time:                        13:33:53   BIC                        1515318.453
Sample:                             4   HQIC                       1515123.159
                              2353832                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.4615      0.019    -24.838      0.000      -0.498      -0.425
pct_blackout.L1     0.1320      0.001    204.551      0.000       0.131       0.133
pct_blackout.L

/Users/rainylty/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/rainylty/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)



Climate Zone: Am
                              ARDL Model Results                              
Dep. Variable:           pct_blackout   No. Observations:               479518
Model:               ARDL(4, 4, 4, 4)   Log Likelihood               29662.794
Method:               Conditional MLE   S.D. of innovations              0.227
Date:                Thu, 24 Oct 2024   AIC                         -59283.587
Time:                        13:36:01   BIC                         -59050.896
Sample:                             4   HQIC                        -59217.600
                               479518                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               1.5435      0.056     27.383      0.000       1.433       1.654
pct_blackout.L1     0.0715      0.001     49.652      0.000       0.069       0.074
pct_blackout.L

ValueError: zero-size array to reduction operation maximum which has no identity

In [5]:

# drop the rows where exog is n/a
df_final_notna = df_final.copy().dropna(subset=['wind_speed', 't2m', 'tp'])

model = ARDL(df_final_notna[['pct_blackout']],
             exog=df_final_notna[['wind_speed', 't2m', 'tp']], lags=1)

model_fit = model.fit()

# print the summary
print(model_fit.summary())

# save the summary
with open(os.path.join(input_dir, 'model_summary.txt'), 'w') as f:
    f.write(model_fit.summary().as_text())

/Users/rainylty/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


                              ARDL Model Results                              
Dep. Variable:           pct_blackout   No. Observations:              7110681
Model:               ARDL(1, 0, 0, 0)   Log Likelihood             -668057.116
Method:               Conditional MLE   S.D. of innovations              0.266
Date:                Thu, 17 Oct 2024   AIC                        1336126.232
Time:                        16:10:03   BIC                        1336208.895
Sample:                             1   HQIC                       1336147.335
                              7110681                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.6713      0.005   -139.258      0.000      -0.681      -0.662
pct_blackout.L1     0.2748      0.000    763.077      0.000       0.274       0.275
wind_speed.L0      -0.0005   7.8

In [18]:
with open(os.path.join(input_dir, 'model_summary.txt'), 'w') as f:
    f.write(model_fit.summary().as_text())

## Interpretation

- Precipitation (tp.L0) is the most significant factor influencing blackout percentages. This makes sense because heavy rain can lead to flooding, infrastructure damage, and other issues that disrupt power systems.

- Temperature (t2m.L0) has a small but positive effect on blackout percentages. High temperatures may increase the load on the power grid (e.g., air conditioning demand), contributing to blackouts.

- Wind speed (wind_speed.L0) has a very minor negative effect, indicating that stronger winds might have a small protective effect against blackouts, possibly due to cooling effects or better air circulation around power equipment.

- Past Blackouts (pct_blackout.L1): The fact that previous blackout percentages have a positive effect suggests that blackouts tend to persist over time. If there’s a blackout in one period, it’s likely to continue to some degree in the next.

In [23]:
import matplotlib.pyplot as plt

# Generate the impulse response for a shock to one of the independent variables (e.g., 't2m')
irf = model.irf_errband_mc(
    steps=10, exog_impulse='t2m', impulse_value=1, repl=1000)

# Plot the impulse response with confidence intervals
fig, ax = plt.subplots(figsize=(10, 6))
irf.plot(ax=ax, impulse='t2m', orth=False)
plt.title('Impulse Response for t2m Shock on pct_blackout')
plt.ylabel('Response of pct_blackout')
plt.xlabel('Time steps')
plt.grid(True)
plt.show()

AttributeError: 'ARDLResults' object has no attribute 'irf_errband_mc'

In [45]:
df_final_geo = pd.read_csv(os.path.join(input_dir, 'df_final_geo_518.csv'))

/var/folders/38/ttqg2y215g16g2ng7jd502_c0000gn/T/ipykernel_88840/1085747112.py:1: DtypeWarning: Columns (8,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df_final_geo = pd.read_csv(os.path.join(input_dir, 'df_final_geo_518.csv'))


In [54]:
# print the statisics into a table, columns as follows:
# : climate zones, number of moniotered stations, number of blackouts, blackout rate, average temperature, average wind speed, average precipitation, average blackout percentage

# create a dictionary to store the statistics
statistics = {}

# loop through each climate zone
for zone in df_final_geo['climate_zone_code_2'].unique():
	# filter the data for the current climate zone
	zone_data = df_final_geo[df_final_geo['climate_zone_code_2'] == zone]
	num_stations = zone_data['station_id'].nunique()
	avg_blackout_rate = zone_data['pct_blackout'].mean()

	# calculate the average temperature
	avg_temp = zone_data['t2m'].mean()

	# calculate the average wind speed
	avg_wind_speed = zone_data['wind_speed'].mean()

	# calculate the average precipitation
	avg_precip = zone_data['tp'].mean()

	# store the statistics in the dictionary
	statistics[zone] = {
		'num_stations': num_stations,
		'avg_blackout_rate': avg_blackout_rate,
		'avg_temp': avg_temp,
		'avg_wind_speed': avg_wind_speed,
		'avg_precip': avg_precip
	}

In [55]:
# pirnt statistics	as a df
statistics_df = pd.DataFrame(statistics).T
# statistics_df.index.name = 'climate_zone_code_2'
# statistics_df.reset_index(inplace=True)

# save the statistics as a csv file
statistics_df


,num_stations,avg_blackout_rate,avg_temp,avg_wind_speed,avg_precip
Aw,126.0,0.028980,298.275138,2.568378,0.001708
BSh,114.0,0.085862,299.258731,2.537897,0.001063
Cw,250.0,0.203950,297.259426,2.145251,0.001807
Am,26.0,0.071880,299.403274,2.156911,0.003444
NaN,0.0,NaN,NaN,NaN,NaN


In [57]:

statistics_df.to_csv(os.path.join(input_dir, 'statistics.csv'), index=True)